<a href="https://colab.research.google.com/github/RayBasilio123/R5/blob/master/WindSun_ARIMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmo ARIMA para Previsão de Séries Temporais com base nos dados do INMET

## Obtendo os dados atráves do github

In [16]:
#imports
import pandas as pd
import numpy as np
from datetime import timedelta
import statsmodels.api as sm
import time

In [17]:
#obtem os dados
url1 = 'https://raw.githubusercontent.com/lrssv/TimeSeriesForecastingWindSun/master/series_ventovel_pampulha_2018_2019'
url2 = 'https://raw.githubusercontent.com/lrssv/TimeSeriesForecastingWindSun/master/series_radiacao_pampulha_2018_2019'
url3 = 'https://raw.githubusercontent.com/RayBasilio123/R5/master/Eto'

df_ventovel = pd.read_csv(url1, header=0, parse_dates=[0], index_col=0, squeeze=True)
df_radiacao = pd.read_csv(url2, header=0, parse_dates=[0], index_col=0, squeeze=True)
df_eto = pd.read_csv(url3, header=0, parse_dates=[0], index_col=0, squeeze=True)

series_ventovel = pd.Series(df_ventovel)
series_radiacao = pd.Series(df_radiacao)
series_eto = pd.Series(df_eto)



In [18]:
series_eto.index.max()

Timestamp('2019-12-31 00:00:00')

## ARIMA 


In [19]:
def run_arima(series, steps_ahead, configuracao):
  result = []
  
  #Lista de data+hora que será previsto
  begin = series.index.max() + timedelta(days=0)
  date_list = [begin + timedelta(hours=x) for x in range(1,steps_ahead+1)]
  
  #Valores da série
  values = series.values

  #ARIMA
  start_fit = time.time()
  mod = sm.tsa.statespace.SARIMAX(values, order=configuracao)
  res = mod.fit(disp=False)
  t_fit = time.time() - start_fit
  
  start_fcast = time.time() 
  forecast = res.forecast(steps=steps_ahead)
  t_fcast = time.time() - start_fcast 
  
  #Resultado no formato para ser exibido no gráfico
  for i in range(steps_ahead):
    if forecast[i] < 0: 
      result.append([date_list[i].strftime('%d/%m/%Y %H:%M:%S'),0])
    else:
      result.append([date_list[i].strftime('%d/%m/%Y %H:%M:%S'),round((forecast[i]),3)])

  return result, t_fit, t_fcast

In [20]:
def run_arimaDay(series, steps_ahead, configuracao):
  result = []
  
  #Lista de data+hora que será previsto
  begin = series.index.max() + timedelta(days=0)
  date_list = [begin + timedelta(days=x) for x in range(1,steps_ahead+1)]
  
  #Valores da série
  values = series.values

  #ARIMA
  start_fit = time.time()
  mod = sm.tsa.statespace.SARIMAX(values, order=configuracao)
  res = mod.fit(disp=False)
  t_fit = time.time() - start_fit
  
  start_fcast = time.time() 
  forecast = res.forecast(steps=steps_ahead)
  t_fcast = time.time() - start_fcast 
  
  #Resultado no formato para ser exibido no gráfico
  for i in range(steps_ahead):
    if forecast[i] < 0: 
      result.append([date_list[i].strftime('%d/%m/%Y '),0])
    else:
      result.append([date_list[i].strftime('%d/%m/%Y '),round((forecast[i]),3)])

  return result, t_fit, t_fcast

Configurações:

In [21]:
order_ven = (12,1,0)

In [22]:
order_rad = (2,1,4) #auto

In [23]:
order_rad = (24,0,24) #graficos
order_eto1 = (6, 0, 41) #graficos
order_eto2 = (5, 1, 2) #graficos

## Previsão 1 hora a frente

In [24]:
result_ven_1 = run_arima(series_ventovel,1,order_ven)
result_ven_1

([['01/01/2020 00:00:00', 2.477]], 6.313431739807129, 0.27041006088256836)

In [25]:
result_rad_1 = run_arima(series_radiacao,1,order_rad)
result_rad_1

/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/statespace/sarimax.py:949: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/statespace/sarimax.py:961: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


([['01/01/2020 00:00:00', 0]], 512.7773995399475, 0.386444091796875)

In [26]:
df_eto_2 = run_arimaDay(series_eto,1,order_eto2)
df_eto_2

/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


([['01/01/2020 ', 5.587]], 1.7943553924560547, 0.010106086730957031)

## Previsão 1 dia a frente

In [27]:
result_ven_1 = run_arima(series_ventovel,24,order_ven)
result_ven_1

([['01/01/2020 00:00:00', 2.477],
  ['01/01/2020 01:00:00', 2.498],
  ['01/01/2020 02:00:00', 2.468],
  ['01/01/2020 03:00:00', 2.48],
  ['01/01/2020 04:00:00', 2.494],
  ['01/01/2020 05:00:00', 2.47],
  ['01/01/2020 06:00:00', 2.445],
  ['01/01/2020 07:00:00', 2.518],
  ['01/01/2020 08:00:00', 2.62],
  ['01/01/2020 09:00:00', 2.675],
  ['01/01/2020 10:00:00', 2.55],
  ['01/01/2020 11:00:00', 2.507],
  ['01/01/2020 12:00:00', 2.485],
  ['01/01/2020 13:00:00', 2.514],
  ['01/01/2020 14:00:00', 2.517],
  ['01/01/2020 15:00:00', 2.519],
  ['01/01/2020 16:00:00', 2.523],
  ['01/01/2020 17:00:00', 2.526],
  ['01/01/2020 18:00:00', 2.528],
  ['01/01/2020 19:00:00', 2.533],
  ['01/01/2020 20:00:00', 2.54],
  ['01/01/2020 21:00:00', 2.541],
  ['01/01/2020 22:00:00', 2.535],
  ['01/01/2020 23:00:00', 2.524]],
 5.972358465194702,
 0.20239520072937012)

In [28]:
result_rad_1 = run_arima(series_radiacao,24,order_rad)
result_rad_1

/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/statespace/sarimax.py:949: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/statespace/sarimax.py:961: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


([['01/01/2020 00:00:00', 0],
  ['01/01/2020 01:00:00', 8.044],
  ['01/01/2020 02:00:00', 37.974],
  ['01/01/2020 03:00:00', 35.903],
  ['01/01/2020 04:00:00', 22.758],
  ['01/01/2020 05:00:00', 0],
  ['01/01/2020 06:00:00', 0],
  ['01/01/2020 07:00:00', 0],
  ['01/01/2020 08:00:00', 44.33],
  ['01/01/2020 09:00:00', 262.261],
  ['01/01/2020 10:00:00', 695.958],
  ['01/01/2020 11:00:00', 1346.744],
  ['01/01/2020 12:00:00', 2078.323],
  ['01/01/2020 13:00:00', 2777.002],
  ['01/01/2020 14:00:00', 3163.522],
  ['01/01/2020 15:00:00', 3165.509],
  ['01/01/2020 16:00:00', 2969.754],
  ['01/01/2020 17:00:00', 2740.491],
  ['01/01/2020 18:00:00', 2338.476],
  ['01/01/2020 19:00:00', 1686.527],
  ['01/01/2020 20:00:00', 942.007],
  ['01/01/2020 21:00:00', 402.353],
  ['01/01/2020 22:00:00', 90.837],
  ['01/01/2020 23:00:00', 0]],
 510.337571144104,
 0.37468862533569336)

In [29]:
df_eto_2 = run_arimaDay(series_eto,1,order_eto2)
df_eto_2

/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


([['01/01/2020 ', 5.587]], 1.8780958652496338, 0.01241445541381836)

## Previsão 7 dias a frente

In [30]:
result_ven_2 = run_arima(series_ventovel,168,order_ven)
result_ven_2

([['01/01/2020 00:00:00', 2.477],
  ['01/01/2020 01:00:00', 2.498],
  ['01/01/2020 02:00:00', 2.468],
  ['01/01/2020 03:00:00', 2.48],
  ['01/01/2020 04:00:00', 2.494],
  ['01/01/2020 05:00:00', 2.47],
  ['01/01/2020 06:00:00', 2.445],
  ['01/01/2020 07:00:00', 2.518],
  ['01/01/2020 08:00:00', 2.62],
  ['01/01/2020 09:00:00', 2.675],
  ['01/01/2020 10:00:00', 2.55],
  ['01/01/2020 11:00:00', 2.507],
  ['01/01/2020 12:00:00', 2.485],
  ['01/01/2020 13:00:00', 2.514],
  ['01/01/2020 14:00:00', 2.517],
  ['01/01/2020 15:00:00', 2.519],
  ['01/01/2020 16:00:00', 2.523],
  ['01/01/2020 17:00:00', 2.526],
  ['01/01/2020 18:00:00', 2.528],
  ['01/01/2020 19:00:00', 2.533],
  ['01/01/2020 20:00:00', 2.54],
  ['01/01/2020 21:00:00', 2.541],
  ['01/01/2020 22:00:00', 2.535],
  ['01/01/2020 23:00:00', 2.524],
  ['02/01/2020 00:00:00', 2.522],
  ['02/01/2020 01:00:00', 2.524],
  ['02/01/2020 02:00:00', 2.527],
  ['02/01/2020 03:00:00', 2.528],
  ['02/01/2020 04:00:00', 2.528],
  ['02/01/2020 05:0

In [31]:
result_rad_2 = run_arima(series_radiacao,168,order_rad)
result_rad_2

/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/statespace/sarimax.py:949: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/statespace/sarimax.py:961: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


([['01/01/2020 00:00:00', 0],
  ['01/01/2020 01:00:00', 8.044],
  ['01/01/2020 02:00:00', 37.974],
  ['01/01/2020 03:00:00', 35.903],
  ['01/01/2020 04:00:00', 22.758],
  ['01/01/2020 05:00:00', 0],
  ['01/01/2020 06:00:00', 0],
  ['01/01/2020 07:00:00', 0],
  ['01/01/2020 08:00:00', 44.33],
  ['01/01/2020 09:00:00', 262.261],
  ['01/01/2020 10:00:00', 695.958],
  ['01/01/2020 11:00:00', 1346.744],
  ['01/01/2020 12:00:00', 2078.323],
  ['01/01/2020 13:00:00', 2777.002],
  ['01/01/2020 14:00:00', 3163.522],
  ['01/01/2020 15:00:00', 3165.509],
  ['01/01/2020 16:00:00', 2969.754],
  ['01/01/2020 17:00:00', 2740.491],
  ['01/01/2020 18:00:00', 2338.476],
  ['01/01/2020 19:00:00', 1686.527],
  ['01/01/2020 20:00:00', 942.007],
  ['01/01/2020 21:00:00', 402.353],
  ['01/01/2020 22:00:00', 90.837],
  ['01/01/2020 23:00:00', 0],
  ['02/01/2020 00:00:00', 0],
  ['02/01/2020 01:00:00', 0],
  ['02/01/2020 02:00:00', 0],
  ['02/01/2020 03:00:00', 3.955],
  ['02/01/2020 04:00:00', 0],
  ['02/01/2

In [32]:
df_eto_2 = run_arimaDay(series_eto,7,order_eto2)
df_eto_2

/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


([['01/01/2020 ', 5.587],
  ['02/01/2020 ', 5.49],
  ['03/01/2020 ', 5.552],
  ['04/01/2020 ', 5.477],
  ['05/01/2020 ', 5.545],
  ['06/01/2020 ', 5.476],
  ['07/01/2020 ', 5.542]],
 1.8100841045379639,
 0.011016130447387695)